# Modelo de Regresión Logística

Este notebook implementa un modelo de regresión logística para predecir la causa de defunción. Se enfoca en las 3 causas más frecuentes del conjunto de datos `defunciones_simplified.csv`.

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

# Cargar los datos procesados
df = pd.read_csv("../defunciones_simplified.csv")

# Seleccionar las 3 causas más comunes
top_3_causas = df['causa'].value_counts().nlargest(3).index
df = df[df['causa'].isin(top_3_causas)]

# Separar variables predictoras y variable objetivo
X = df.drop(columns=["causa"])
y = df["causa"]

# Detectar columnas categóricas
cat_features = X.select_dtypes(include='object').columns.tolist()

# Dividir datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Crear pipeline de preprocesamiento + modelo
preprocessor = ColumnTransformer([
    ('cat', OneHotEncoder(handle_unknown='ignore'), cat_features)
])

pipeline = Pipeline([
    ('preprocess', preprocessor),
    ('classifier', LogisticRegression(max_iter=1000))
])

# Entrenar modelo
pipeline.fit(X_train, y_train)

# Hacer predicciones y evaluar
y_pred = pipeline.predict(X_test)
print(classification_report(y_test, y_pred))

                                                                precision    recall  f1-score   support

Diabetes mellitus no especificada, sin mención de complicación       0.42      0.32      0.37      9352
          Infarto agudo del miocardio, sin otra especificación       0.50      0.83      0.63     17193
                                     Neumonía, no especificada       0.98      0.27      0.42     12507

                                                      accuracy                           0.53     39052
                                                     macro avg       0.64      0.47      0.47     39052
                                                  weighted avg       0.64      0.53      0.50     39052

